In [109]:
import numpy as np
import sys
sys.path.append(r"C:\Users\gapar\Logical-Clifford-Synthesis")
from Algorithms.algorithm_1 import SymplecticOperations
from helper_functions.helperfunctions import SymplecticMatrix
%load_ext autoreload
%autoreload 2


# Initialize the SymplecticOperations class
symplectic_ops = SymplecticOperations()
def gf2matinv(matrix):
    """
    Compute the inverse of a matrix in GF(2) using Gaussian elimination.
    """
    matrix = np.array(matrix, dtype=np.int8) % 2  # Ensure the matrix is in GF(2) and explicitly set to an integer type
    m = len(matrix)
    aug_matrix = np.concatenate((matrix, np.eye(m, dtype=np.int8)), axis=1)  # Explicit dtype
    for col in range(m):
        for row in range(col, m):
            if aug_matrix[row, col]:
                if row != col:
                    # Swap rows
                    aug_matrix[[col, row]] = aug_matrix[[row, col]]
                break
        else:  # This else belongs to the for, executed only if no break occurred
            raise ValueError("Matrix is singular over GF(2)")

        for i in range(m):
            if i != col and aug_matrix[i, col]:
                # Use explicit type conversion if necessary
                aug_matrix[i] = (aug_matrix[i] + aug_matrix[col]) % 2  # This replaces the ^= operation for clarity and compatibility
    return aug_matrix[:, m:]




def bi2de(b):
    """ Convert binary matrix to decimal. """
    b = np.array(b)  # Ensure input is a NumPy array
    return b.dot(1 << np.arange(b.shape[-1])[::-1])

def symplectic_code(U, V):

    m = U.shape[1] // 2
    print('m =', m)
    k = 2 * m - V.shape[0]
    print('k =', k)
    tot = 2 ** (k * (k + 1) // 2)
    print('tot =', tot)
    F_all = [None] * tot

    # Find one solution using symplectic transvections
    F0, Transvecs = symplectic_ops.find_symp_mat_transvecs(U[0:(2*m-k), :], V)
    print("f0 =", F0)
    #F0 = find_symp_mat_transvecs(U[0:(2*m-k), :], V)

    A = U.dot(F0) % 2
    print("A =", A)
    Ainv = gf2matinv(A)
    print("Ainv =", Ainv)
    Basis = np.vstack((A[(m - k):m, :], A[(2*m - k ):(2*m), :]))
    print("basis =", Basis)
    Subspace = np.mod(np.dot(np.array([list(np.binary_repr(i, 2*k)) for i in range(2**(2*k))], dtype=int), Basis), 2)
    print("subspace")
    print(Subspace)
    StabF0 = A[(m - k):m, :]
    print("StabF0")
    print(StabF0)
    Choices = [None] * k
    print("Choices")
    print(Choices)

    # Calculate all choices for each vector in Subspace using just the conditions
    # imposed by the (modified) stabilizer generators, i.e., the rows of S*F0
    for i in range(k):
        # Impose symplectic inner product of 1 with the "fixed" stabilizer
        h = np.concatenate((np.zeros(i, dtype=int), [1], np.zeros(k - i-1, dtype=int)))
        print("h")
        print(h)
        Innpdts = np.dot(Subspace, np.fft.fftshift(StabF0, axes=1).T) % 2
        print("Innpdts")
        print(Innpdts)
        Choices[i] = Subspace[np.all(Innpdts == h, axis=1)]
    print("Choices")
    print(Choices)
    # First free vector, i.e. row 1 of Subspace has 2^k choices,
    # second free vector has 2^(k-1) choices and so on...
    for l in range(tot):
        Bl = np.copy(A)
        print("Bl")
        print(Bl)
        V = np.zeros((k, 2 * m), dtype=int)
        print("V")
        print(V)
        lbin = np.array(list(np.binary_repr(l, width=k*(k+1)//2)), dtype=int)
        print("lbin")
        print(lbin)
        v1_ind = int(''.join(map(str, lbin[:k])), 2)
        print("v1_ind")
        print(v1_ind)
        V[0, :] = Choices[0][v1_ind]
        for i in range(1, k):
            # Translating the sum and range generation from MATLAB to Python
            vi_ind = int(''.join(map(str, lbin[sum(range(k, k-i, -1)):(sum(range(k, k-i, -1))+k-i)])), 2)
            print("vi_ind")
            print(vi_ind)
            Innprods = np.dot(Choices[i], np.fft.fftshift(V, axes=1).T) % 2
            
            # Impose symplectic inner product of 0 with chosen free vectors
            Ch_i = Choices[i][np.all(Innprods == 0, axis=1)]
            V[i, :] = Ch_i[vi_ind]
        Bl[(2 * m - k):(2 * m), :] = V
        F = np.dot(Ainv, Bl) % 2
        F_all[l] = np.dot(F0, F) % 2

    return F_all


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
X = np.array([[1, 0,0,0], [0, 1,0,0],[0,0,1,0],[0,0,0,1]])
Y = np.array([[0,0,1,0],[0,0,1,1]])
symplectic_code(X,Y)



m = 2
k = 2
tot = 8
f0 = [[0. 0. 1. 0.]
 [0. 0. 1. 1.]
 [1. 1. 1. 1.]
 [0. 1. 1. 0.]]
A = [[0. 0. 1. 0.]
 [0. 0. 1. 1.]
 [1. 1. 1. 1.]
 [0. 1. 1. 0.]]
Ainv = [[1 1 1 1]
 [1 0 0 1]
 [1 0 0 0]
 [1 1 0 0]]
basis = [[0. 0. 1. 0.]
 [0. 0. 1. 1.]
 [1. 1. 1. 1.]
 [0. 1. 1. 0.]]
subspace
[[0. 0. 0. 0.]
 [0. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 0. 0. 1.]
 [0. 0. 1. 1.]
 [0. 1. 0. 1.]
 [1. 1. 0. 0.]
 [1. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 1. 0. 1.]
 [1. 0. 1. 1.]
 [0. 0. 0. 1.]
 [0. 1. 1. 1.]
 [1. 1. 1. 0.]
 [1. 0. 0. 0.]]
StabF0
[[0. 0. 1. 0.]
 [0. 0. 1. 1.]]
Choices
[None, None]
h
[1 0]
Innpdts
[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
h
[0 1]
Innpdts
[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Choices
[array([[1., 1., 1., 1.],
       [1., 1., 0., 0.],
       [1., 1., 0., 1.],
       [1., 1

[array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 1., 1., 0.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 1., 0., 1.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 0., 0.],
        [0., 1., 0., 0.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 0., 0.],
        [0., 1., 1., 1.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 0., 1.],
        [0., 1., 1., 0.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 0., 1.],
        [0., 1., 0., 1.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 1., 0.],
        [0., 1., 0., 0.]]),
 array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 1., 0.],
        [0., 1., 1., 1.]])]